**Extracción y carga de los archivos desde una carpeta contenida en Google Drive:** Esta sección conecta el Google Drive con Colab para acceder a los archivos contenidos en dicha carpeta, específicamente al archivo train.csv que contiene los datos de estudiantes y sus resultados en las pruebas. Luego se usa la librería pandas para leer ese archivo y almacenarlo en una estructura de tabla que facilita el análisis de datos.
También se importan herramientas de preprocesamiento de scikit-learn como OneHotEncoder que se usa para convertir variables categóricas en columnas binarias (0 y 1) y MinMaxScaler para escalar los valores numéricos al rango de 0 a 1.

In [4]:
# Conexión a Google Drive y carga de datos
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler


file_path = '/content/drive/MyDrive/Proyecto IA/Pruebas_Saber_Pro/train.csv'
df = pd.read_csv(file_path)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Preprocesamiento de datos:**
Se realiza el preprocesamiento del conjunto de datos del archivo train.csv con el fin de preparar la información para un modelo de clasificación. Primero se identifican y se tratan los valores nulos, en las columnas categóricas se reemplazaron por la palabra **Desconocido** y en las columnas numéricas por su mediana, también se eliminan las filas sin etiqueta de rendimiento.

Después se transforman las variables categóricas (como modalidad de estudio o tipo de institución) en variables numéricas mediante codificación One-Hot lo que permite usarlas en algoritmos de aprendizaje automático y por ultimo se normalizan las variables numéricas para que sus valores estuvieran en una misma escala entre 0 y 1,esto permitira mejorar el rendimiento y la estabilidad de los modelos.


In [5]:
#Muestra dimensiones iniciales y columnas con valores nulos
print("Dimensiones del dataset:", df.shape)
print("\nColumnas con valores nulos:")
print(df.isnull().sum()[df.isnull().sum() > 0])

#Corrobora que la etiqueta no tenga nulos
df = df.dropna(subset=['RENDIMIENTO_GLOBAL'])

#Imputación de valores faltantes
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('Desconocido')  #Rellena con 'Desconocido' si es categórica
    else:
        df[col] = df[col].fillna(df[col].median())  #Rellena con mediana si es numérica

#Separa las variables categóricas para codificar pero excluye el ID y el rendimiento global
columnas_categoricas = df.select_dtypes(include='object').columns.tolist()
if 'RENDIMIENTO_GLOBAL' in columnas_categoricas:
    columnas_categoricas.remove('RENDIMIENTO_GLOBAL')
if 'ID' in columnas_categoricas:
    columnas_categoricas.remove('ID')

#Uso de One-Hot para variables categóricas
df_codificado = pd.get_dummies(df, columns=columnas_categoricas)

#Normalización de las variables numéricas pero no tiene en cuenta el ID
columnas_numericas = df_codificado.select_dtypes(include=['int64', 'float64']).columns.tolist()
if 'ID' in columnas_numericas:
    columnas_numericas.remove('ID')

escalador = MinMaxScaler()
df_codificado[columnas_numericas] = escalador.fit_transform(df_codificado[columnas_numericas])

#Muestra las dimensiones finales y vista previa
print("\nDimensiones después del preprocesamiento:", df_codificado.shape)
df_codificado.head()

Dimensiones del dataset: (692500, 21)

Columnas con valores nulos:
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_TIENEINTERNET.1              26629
FAMI_EDUCACIONMADRE               23664
dtype: int64

Dimensiones después del preprocesamiento: (692500, 1055)


,ID,PERIODO,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4,ESTU_PRGM_ACADEMICO_3° CICLO PROFESIONAL NEGOCIOS INTERNACIONALES,ESTU_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,ESTU_PRGM_ACADEMICO_ACUICULTURA,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,904256,0.966667,medio-alto,0.490107,0.427105,0.968750,0.804217,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,645256,0.966667,bajo,0.473364,0.441478,0.912500,0.795181,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,308367,0.666667,bajo,0.452055,0.439425,0.953125,0.795181,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,470353,0.400000,alto,0.738204,0.353183,0.787500,0.572289,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,989032,0.966667,medio-bajo,0.480974,0.476386,0.890625,0.885542,False,False,False,...,False,False,False,False,True,False,False,False,False,False
